In [1]:
# import libraries
import numpy as np
import pandas as pd
import pysindy as ps
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error

from dataPrep import DataPreparation

In [2]:
drivingProfile = pd.read_csv('./parameter.csv')
data = pd.read_csv('./HighFrq/HighFrq_input.csv')

In [3]:
dp = DataPreparation(drivingProfile)

Time = data['Time'].to_numpy()
deltaSteer = data['Drv_DeltaSteer'].to_numpy()
Vx = data['Veh_Vx'].to_numpy()

In [4]:
[time, output] = dp.getOutput(Time, deltaSteer, Vx)
[time, inputs] = dp.getInput(Time, deltaSteer, Vx)
[time, states] = dp.getStates(Time, deltaSteer, Vx)
output = np.transpose(np.array(output))
inputs = np.transpose(np.array(inputs))
states = np.transpose(np.array(states))
time = np.array(time)

In [5]:
inputs

array([[1.94444444e+01, 0.00000000e+00, 0.00000000e+00],
       [1.94361107e+01, 0.00000000e+00, 0.00000000e+00],
       [1.94075411e+01, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.50346177e+00, 8.86579672e-01, 5.99567657e+05],
       [4.48990527e+00, 6.07229047e+00, 1.09316250e+05],
       [4.47643096e+00, 4.63501995e-01, 8.45712631e+05]])

In [6]:
states_mod = np.where(states == 0, 1e-8, states)
inputs_mod = np.where(inputs == 0, 1e-8, inputs)

In [7]:
for el in inputs:
    print(el)

[19.44444444  0.          0.        ]
[19.4361107  0.         0.       ]
[19.40754113  0.          0.        ]
[19.36526168  0.          0.        ]
[19.32718332  0.          0.        ]
[19.29410943  0.          0.        ]
[19.26058862  0.          0.        ]
[19.22573375  0.          0.        ]
[19.19093674  0.          0.        ]
[19.15651394  0.          0.        ]
[19.1221354  0.         0.       ]
[19.08765416  0.          0.        ]
[19.05316034  0.          0.        ]
[19.01871103  0.          0.        ]
[18.98429521  0.          0.        ]
[18.94990462  0.          0.        ]
[18.91554959  0.          0.        ]
[18.88124008  0.          0.        ]
[18.8469787  0.         0.       ]
[18.81276396  0.          0.        ]
[18.77859604  0.          0.        ]
[18.74447582  0.          0.        ]
[18.71040489  0.          0.        ]
[18.6763821  0.         0.       ]
[18.64240484  0.          0.        ]
[18.60847064  0.          0.        ]
[18.57457797  0.        

In [8]:
np.array(states)

array([[ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       ...,
       [ 3.47908347, 14.80356019],
       [ 1.26628866,  7.29845604],
       [ 3.54264795, 10.94856341]])

In [10]:
feature_lib = ps.PolynomialLibrary(degree=2, include_bias=False)

params = ps.CustomLibrary(library_functions=[
    # lambda x: ((1/x) + (1/(x*x))),
    # lambda x: 1/x,
    lambda x: 1/x,
    lambda x: x,
    lambda x: x
], function_names=[
    # lambda x: x+'^-1'+x+'^-2',
    # lambda x: x+'^-1',
    lambda x: x+'^-1',
    lambda x: x,
    lambda x: x
], include_bias=True)


lib = ps.ParameterizedLibrary(
    feature_library=feature_lib,
    parameter_library=params,
    num_features=3,
    num_parameters=2,
)

optimizer_stable = ps.StableLinearSR3(
    threshold=0.0,
    thresholder='l1',
    nu=1e-4,
    max_iter=1000,
    tol=1e-5,
    verbose=False,
)


opt = ps.STLSQ(threshold=1e-4, normalize_columns=False)
model = ps.SINDy(feature_library=lib, optimizer=opt, feature_names=["x1", "x2", "v", "u1", "u2"], discrete_time=True)

model.fit(np.array(states_mod), u=np.array(inputs_mod))
model.print()

(x1)[k+1] = 1.130 1 x1[k] + -0.043 1 x2[k] + -0.124 1 v[k] + -0.168 1 x1[k] x2[k] + 0.020 1 x2[k]^2 + 0.009 1 v[k]^2 + -0.280 u1[k]^-1 x1[k] + 0.044 u1[k]^-1 x2[k] + 0.005 u1[k]^-1 v[k] + 0.095 u1[k]^-1 x1[k]^2 + -0.030 u1[k]^-1 x1[k] x2[k] + 0.033 u1[k]^-1 x1[k] v[k] + 0.002 u1[k]^-1 x2[k]^2 + -0.005 u1[k]^-1 x2[k] v[k] + 156.745 u2[k]^-1 x1[k] + -156.509 u2[k]^-1 x2[k] + -0.005 u2[k]^-1 v[k] + 0.761 u2[k]^-1 x1[k]^2 + -0.076 u2[k]^-1 x1[k] x2[k] + 1.821 u2[k]^-1 x1[k] v[k] + -0.556 u2[k]^-1 x2[k]^2 + 13.257 u2[k]^-1 x2[k] v[k] + 0.276 u1[k] x1[k] + -0.039 u1[k] x2[k] + 0.067 u1[k] v[k] + -0.052 u1[k] x1[k]^2 + 0.015 u1[k] x1[k] x2[k] + -0.037 u1[k] x1[k] v[k] + -0.001 u1[k] x2[k]^2 + 0.004 u1[k] x2[k] v[k] + -0.004 u1[k] v[k]^2 + 0.002 u2[k] x2[k]^2 + 0.013 u2[k] v[k]^2 + 0.276 u1[k] x1[k] + -0.039 u1[k] x2[k] + 0.067 u1[k] v[k] + -0.052 u1[k] x1[k]^2 + 0.015 u1[k] x1[k] x2[k] + -0.037 u1[k] x1[k] v[k] + -0.001 u1[k] x2[k]^2 + 0.004 u1[k] x2[k] v[k] + -0.004 u1[k] v[k]^2 + -0.002 u2[